In [41]:
import pandas as pd
import pandas_profiling
import numpy as np
import time
import glob
import csv
import operator

## Normalize the dataset units and property names as suggested by paul which was then modified by Meera in all_v2.csv. 

In [42]:
#Reads variables we are interetsed in prop_map
df = pd.read_csv("Characteristic_Names_Map_Match.csv", encoding='UTF-8')
prop_map = {}
for (idx,  row) in df.iterrows():
    property1 = row.loc['CharacteristicName_1'].strip()
    property2 = row.loc['CharacteristicName_2'].strip()
    prop_map[property1] = property2;
    prop_map[property2] = property2;
df = pd.read_csv("Interested_Variables.csv", encoding='UTF-8')
for (idx,  row) in df.iterrows():
    property1 = row.loc['CharacteristicName'].strip()
    if(property1 not in prop_map):
        prop_map[property1] = property1;

In [43]:
orig_unit_column = 'ResultMeasure/MeasureUnitCode'
orig_prop_column = 'CharacteristicName'
# creates a map with key = <CharacteristicName, ResultMeasure/MeasureUnitCode> , 
# value = row in all_v2.csv which has transformation functions
df = pd.read_csv('all_v2.csv', encoding='UTF-8')
func_map = {}
for (idx, row) in df.iterrows():
    key = str(row['CharacteristicName']).strip()+","+str(row['ResultMeasure/MeasureUnitCode']).strip()
    value = row
    func_map[key] = value;

In [ ]:
# Time to normalize the dataset, so read the files. 
# To output the normalized dataset we assume a file structure similar to SITES_DATA directory but with 
# directory name MODIFIED_SITES_DATA. Easiest way to create this file structure is  to make a copy of SITES_DATA 
# and name it as MODIFIED_SITES_DATA and then remove all the already present CSVs containing measurements using
# ‘for f in MODIFIED_SITES_DATA/*/WB_CHUNKS/*/SITES/*.csv; do rm "$f"; done’. This command may take some time to run.

allFiles = glob.glob("SITES_DATA/*/WB_CHUNKS/*/SITES/*.csv")

In [ ]:
i = 0
anomalous_values = 0;
# normalize the dataset
for file_ in allFiles:
    df = pd.read_csv(file_, encoding='UTF-8')
    if(orig_prop_column not in df.columns):
        continue;
    # create extra columns with bogus default values
    df['Standard unit'] = df['CharacteristicName']
    df['Transformation'] = df['CharacteristicName']
    df['Notes'] = df['CharacteristicName']
    df['Action'] = df['CharacteristicName']
    df['Function'] = df['CharacteristicName']
    df['NormalizedResultMeasureValue'] = df['CharacteristicName'];
    row_list = [];
    anomaly = False;
    for (idx, row) in df.iterrows():
        key = str(row[orig_prop_column]).strip();
        if(key not in prop_map):
            continue;
        # change CharacteristicName to normalized one
        row[orig_prop_column] = prop_map[row[orig_prop_column]]
        key = (str(row['CharacteristicName']).strip())+","+(str(row['ResultMeasure/MeasureUnitCode']).strip())
        if key in func_map:
            n_row = func_map[key];
            # Fill extra column values replacing the bogus ones
            row['Standard unit'] = n_row['Standard unit']
            row['Transformation'] = n_row['Transformation']
            row['Notes'] = n_row['Notes']
            row['Action'] = n_row['Action']
            row['Function'] = n_row['Function']
            row['NormalizedCharacteristicName'] = row['CharacteristicName'];
            row['NormalizedResultMeasureValue'] = row['ResultMeasureValue'];
            row['NormalizedResultMeasure/MeasureUnitCode'] = row['ResultMeasure/MeasureUnitCode'];
            #If action is keep, no need to modify anything
            if(n_row['Action'] == 'keep'):
                row_list.append(row);
            # this means its a convert, so some normalization need to happen
            elif(n_row['Action'] != 'delete'):
                exec(row['Function']); # Function is defined in all_v2.csv
                try:
                    # Might fail because of invalid data values such as "ND, *ND, *0.15, NRR, Not detected, etc"
                    row['ResultMeasureValue'] = float(row['ResultMeasureValue'])
                    result = (f(row));
                except:
                    print(row['ResultMeasureValue'])
                    anomalous_values = anomalous_values + 1;
                    anomaly = True
                    continue;
                
                row['NormalizedCharacteristicName'] = result[0]
                row['NormalizedResultMeasureValue'] = result[1]
                row['NormalizedResultMeasure/MeasureUnitCode'] = result[2]
                # store the row in row_list which will directly be printed to a file after the loop
                row_list.append(row)
    if(anomaly == True):
        print(file_)
    #Create dataframe from normalized rows
    new_df = pd.DataFrame(row_list)
    newFileName = "MODIFIED_SITES_DATA/"+file_.split("/",1)[1];
    i = i+1
    #Print normalized table to file
    new_df.to_csv(newFileName,index=False)
print("Anomalous_values", anomalous_values);

*0.004
ND
*371.
*0.007
ND
*10.39
ND
*7.88
*0.006
ND
ND
SITES_DATA/INDIANA_SITES/WB_CHUNKS/IN_WB_0/SITES/IN_WB_151959502_SITES.csv
8.0
SITES_DATA/INDIANA_SITES/WB_CHUNKS/IN_WB_0/SITES/IN_WB_152274814_SITES.csv
*5.
ND
*0.513
*8.
*2.4
*0.90
ND
*1.2
ND
*1.5
*4.1
ND
*0.540
*1.4
*0.42
*<1.0
*0.002
*1.5
*1.5
*0.590
*<1
*0.44
ND
*<1.0
SITES_DATA/INDIANA_SITES/WB_CHUNKS/IN_WB_0/SITES/IN_WB_165384840_SITES.csv
*0.004
ND
*371.
*0.007
ND
*10.39
ND
SITES_DATA/INDIANA_SITES/WB_CHUNKS/IN_WB_0/SITES/IN_WB_165384841_SITES.csv


/Users/deepu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (20,25,26,41,42,48,49,55,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*0.009
*4.41
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_123148000_SITES.csv
**
*1.93
ND
*1.1
*0.02
*2.22
ND
*2.36
*2.26
*2.42
ND
*1.76
*1.2
**
*0.017
ND
*0.013
*0.015
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_123148039_SITES.csv
*0.016
*0.017
ND
*0.018
ND
*0.012
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_123148133_SITES.csv
*3.4
*2.6
*4.0
*2.6
*0.014
ND
*2.2
*2.1
*0.009
*4
*0.022
*0.012
*2.31
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_123148135_SITES.csv
*1.65
*2.37
*4.05
*2.32
*14.5
*2.22
<1.
**
*<1
*2.2
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_123398250_SITES.csv


/Users/deepu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (25,26,31,42,54,55,57,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*Non-detect
*Non-detect
*Non-detect
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_155410966_SITES.csv
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Non-detect
*Non-detect
*Non-detect
*Non-detect
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
*Not Detected
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_155417357_SITES.csv
ND
*1.05
*0
*1.02
ND
ND
*1.65
*1.46
*2.82
*0.018
*1.73
**
*1.62
*1.08
*0.008
*1.34
*0.010
*1.23
*1.61
*1.6
*1.1
*1.8
*1.8
*0.011
*0.014
**
*1.2
<1.
*1.8
*1.7
**
*0.008
**
ND
ND
**
ND
ND
*2.10
*2.77
*1.46
*0.013
*0.015
*0.010
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_69884134_SITES.csv
**
*4.81
*4.8
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_69884394_SITES.csv
*0.030
*0.023
SITES_DA

...

ND
ND
ND
ND
<0.02
ND
<0.02
*1.3
*0.316
**
ND
64.5F
<0.02
<0.02
*2.15
*2.32
*0.0100
*0.0870
*0.0130
*0.0350
ND
ND
ND
*1.8
*0.11
*0.35
ND
*0
ND
ND
**
**
**
*0.001
ND
ND
ND
ND
*1.05
*0.035
*0.065
*4.55
ND
ND
*29.3
*0.37
ND
ND
*27.1
*3.85
*0.010
*0.066
*3.31
*0.011
*2.38
*0.082
*3.14
*11.8
ND
ND
*<4.5
*0.038
*0.016
*3
*1.9
*2.9
*0.025
*0.017
*6.
*0.021
**
**
*<0.034
*0.349
*0.050
*0.359
*0.019
*0.027
*0.037
*21.0
*8.68
*0.019
*0.033
*8.21
ND
*0.019
*0.020
ND
ND
ND
ND
*0.034
ND
*2.27
*0.018
ND
ND
*ND
ND
<10.
*ND
*4.09
*0.015
*0.030
*0.016
ND
*0.033
ND
ND
ND
*1.3
<0.015
**
**
**
**
**
**
<0.02
<0.02
<0.02
*0.039
*0.028
**
**
**
**
**
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_70332997_SITES.csv
ND
*2.53
ND
<5.
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_70333153_SITES.csv
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_70333483_SITES.csv
*0.035
*26.
*5
*0.030
*0.051
*2.85
*11.5
*0.043
*0.023
*0.003
ND
ND
*0.8

/Users/deepu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,23,25,26,31,42,52,56,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*0.014
*2.18
*2.5
*2.89
*0.013
*0.019
*2.01
*2.84
*1.2
*3.1
*0.015
ND
ND
ND
*2.2
ND
*10.
*<4.5
*3.6
*4.2
*0.016
*2.8
*0.026
*0.018
ND
*0.088
*2.8
*0.023
*0.211
*0.009
*3.2
*1.6
*0.018
*4.8
*0.022
ND
*2
ND
*<1
ND
*2.45
*4.84
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
ND
*0.025
*0.028
*0.016
*0.018
*0.023
*0.015
*8.58
*0.020
*0.021
*0.014
*0.016
*ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_70338361_SITES.csv
*3
ND
*3.05
*0.29
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_73027855_SITES.csv
ND
ND
ND
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_73028737_SITES.csv
*35.
*14.7
ND
*4.
*6.67
*250
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_73028813_SITES.csv
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_75473461_SITES.csv
*0.009
*1.4
*0.010
*0.38
ND
ND
SITES_DATA/MICHIGAN_SITES/WB_CHUNKS/MI_WB_0/SITES/MI_WB_75473481_SITES.csv
*0.034
*11.2
*0.018
*0.030
*4.27
*2.79
*2.39
*0.026
*2.76
*2.64
**
*1.03
*0.021
*3.2
*0.013
ND
SITES_DATA/MICHIGAN_SITES

/Users/deepu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (31,36,38,40,50,55,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
no result reported
no result reported
no result reported
no result reported
no result reported
no result reported
no result reported
No result recorded.
No result recorded.
no result reported
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result reported.
No result reported.
No result reported.
NRR
No result reported.
No result reported.
No result reported.
No result reported.
No result reported.
No result reported.
No result reported.
No result reported.
no result reported
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
NRR
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No result recorded.
No re